In [ ]:
library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(magrittr)
library(padr)
library(kableExtra)
library(purrr)
library(ggplot2)
library(ggthemr)
library(here)
library(lubridate)
# library(caret)
# library(scales)
ggthemr("fresh")
devtools::load_all()

In [ ]:
# The following types are being modelled:
# Erle - Alder - Aulne - Alnus
# Birke - Birch - Bouleau - Betula
# Gräser - Grasses - Graminées - Poaceae
# Ambrosia - Ragweed - Ambroisie - Ambrosia

species_all <- tibble(
  taxon = c(
    "Castanea",
    "Alnus",
    "Ulmus",
    "Cupressus",
    "Fraxinus",
    "Fagus",
    "Juglans",
    "Plantago",
    "Corylus",
    "Pinus",
    "Quercus",
    "Rumex",
    "Platanus",
    "Populus",
    "Poaceae",
    "Salix",
    "Betula",
    "Carpinus",
    "Urtica",
    "Taxus",
    "Picea",
    "Ambrosia"
  ),
  hirst_taxon = c(
    "kacasth0",
    "kaalnuh0",
    "kaulmuh0",
    "kacuprh0",
    "kafraxh0",
    "kafaguh0",
    "kajuglh0",
    "khplanh0",
    "kacoryh0",
    "kapinuh0",
    "kaquerh0",
    "khrumeh0",
    "kaplath0",
    "kapopuh0",
    "khpoach0",
    "kasalih0",
    "kabetuh0",
    "kacarph0",
    "khurtih0",
    "kataxuh0",
    "kapiceh0",
    "khambrh0"
  ),
  cosmo_taxon = c(
    NA_character_,
    "ALNU",
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    "POAC",
    NA_character_,
    "BETU",
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    "AMBR"
  ),
  fieldextra_taxon = c(
    NA_character_,
    "ALNU24",
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    "POAC24",
    NA_character_,
    "BETU24",
    NA_character_,
    NA_character_,
    NA_character_,
    NA_character_,
    "AMBR24"
  )
)

species <- species_all %>%
  filter(taxon %in% c("Alnus", "Ambrosia", "Betula", "Poaceae"))

stations <-
  tibble(
    hirst_station = c(
      "PDS",
      "PBU",
      "PMU",
      "PBS",
      "PZH",
      "PLZ",
      "PBE",
      # "PPY",
      "PNE",
      "PVI",
      "PLS",
      "PGE",
      "PCF",
      "PLO",
      # "BLR",
      "PLU"
    ),
    station = c(
      "Wolfgang",
      "Buchs",
      "Münsterlingen",
      "Basel",
      "Zürich",
      "Luzern",
      "Bern",
      # "Payerne",
      "Neuchâtel",
      "Visp",
      "Lausanne",
      "Genève",
      "La-Chaux-de-Fonds",
      "Locarno",
      # "Balerna",
      "Lugano"
    ),
    cosmo_station = c(
      "CHDAVO",
      "CHBUCH",
      "CHMUEN",
      "CHBASE",
      "CHZUER",
      "CHLUZE",
      "CHBERN",
      # NA_character_,
      "CHNEUC",
      "CHVISP",
      "CHLAUS",
      "CHGENE",
      "CHLACH",
      "CHLOCA",
      # NA_character_,
      "CHLUGA"
    )
  ) %>%
  arrange(hirst_station)

In [ ]:
data_dwh <- import_data_dwh(paste0(here(), "/data/dwh/pollen_dwh_daily.txt"))
  

In [ ]:


data_dwh <- import_data_dwh(paste0(here(), "/data/dwh/pollen_dwh_daily.txt"))

data_alnu20 <- import_data_cosmo("/scratch/sadamov/wd/20_alnu_pheno_v2/mod_pollen_combined.txt", type = "Phenology V2")
data_alnu20 %<>% mutate(value = 0.6 * value)
data_alnu19 <- import_data_cosmo("/scratch/sadamov/wd/19_alnu_pheno_v2/mod_pollen_combined.txt", type = "Phenology V2")
data_alnu19 %<>% mutate(value = 0.6 * value)
data_betu20 <- import_data_cosmo("/scratch/sadamov/wd/20_betu_pheno_v2/mod_pollen_combined.txt", type = "Phenology V2")
data_betu19 <- import_data_cosmo("/scratch/sadamov/wd/19_betu_pheno_v2/mod_pollen_combined.txt", type = "Phenology V2")
data_poac20 <- import_data_cosmo("/scratch/sadamov/wd/20_poac_pheno_v2/mod_pollen_combined.txt", type = "Phenology V2")
data_poac19 <- import_data_cosmo("/scratch/sadamov/wd/19_poac_pheno_v2/mod_pollen_combined.txt", type = "Phenology V2")

data_list <- list(
  alnu20 = data_alnu20,
  alnu19 = data_alnu19,
  betu20 = data_betu20,
  betu19 = data_betu19,
  poac20 = data_poac20,
  poac19 = data_poac19
)

data_daily_list <- map(data_list, ~ .x %>%
  aggregate_pollen() %>%
  impute_daily() %>%
  filter(measurement == "concentration"))

data_daily <- data_daily_list %>% bind_rows()

data_dwh %<>% impute_daily()

data_daily_comp <- data_dwh %>%
  select(date, taxon, station, obs = value) %>%
  right_join(data_daily, by = c("taxon", "date", "station")) %>%
  select(date, taxon, station, obs, value)

In [ ]:
stations$station
pol_tuning <- tibble(taxon = rep(c("Alnus", "Betula", "Poaceae"), each = 14),
  tuning_orig = rep(c(0.34, 0.63, 0.26), each = 14),
  station = rep(stations$station, times = 3))

data_daily_comp %>%
  group_by(station, taxon) %>%
  summarise(obs = sum(obs, na.rm = TRUE),
            value = sum(value, na.rm = TRUE)) %>%
  ungroup() %>%
  mutate(tuning = obs / value) %>%
  left_join(pol_tuning, by = c("station", "taxon")) %>%
  mutate(tuning_new = tuning * tuning_orig) %>%
  arrange(taxon, station)